<a href="https://colab.research.google.com/github/dbstj1231/2023_AI_Academy_ASR/blob/main/6_nemo_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. NeMo fine-tuning

## NeMo (NVIDIA, 2019)
-  https://developer.nvidia.com/nvidia-nemo (homepage)
- https://github.com/NVIDIA/NeMo (source)
- open-source framework for developers to build and train state-of-the-art (SOTA) conversational AI models
- PyTorch, PyTorch Lightning을 기반으로 작성된 E2E toolkit
- SOTA model들의 pretrained model을 제공
- https://catalog.ngc.nvidia.com/ (ASR pretrained models)

## NeMo 설치

### pip를 이용한 설치
- pip install nemo_toolkit['all']

### Source code를 이용한 설치
- apt-get update && apt-get install -y libsndfile1 ffmpeg
- git clone https://github.com/NVIDIA/NeMo
- cd NeMo
- ./reinstall.sh

### NeMo Docker containers
- https://catalog.ngc.nvidia.com/orgs/nvidia/containers/nemo
- docker pull nvcr.io/nvidia/nemo:22.11 (build container)
- docker run --runtime=nvidia -it --rm -v --shm-size=16g -p 8888:8888 -p 6006:6006 --ulimit memlock=-1 --ulimit stack=67108864 nvcr.io/nvidia/nemo:22.11 (docker run)

In [ ]:
# install NeMo
!pip install git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all]

- `omegaconf` : yaml, json등의 configuration 파일을 읽고 쓸 수 있는 라이브러리, Nemo에서 기본적으로 사용
- `nemo.collections.asr` : NeMo ASR Class
- `nemo.utils.exp_manager` : 학습로그, conf 등에 사용되는 라이브러리

In [ ]:
from omegaconf import OmegaConf, open_dict

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr
from nemo.utils import exp_manager

## pre-trained 모델 불러오기
- Quartznet model 사용
- https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/asr/models.html#quartznet
- SOTA성능을 보이는 모델은 아니지만, 파라미터 수가 작아 low resourced computing 환경에서 동작이 쉬움
- 500MB~1,000MB 사이의 SOTA 수준 모델에 비해 78MB의 적은 용량으로 동작 가능
- Librispeech test set 기준 4.19%의 word error rate (WER)
- https://arxiv.org/abs/1910.10261 (paper)


<img src = "https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/_images/quartz_vertical.png" height=700>

- https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/stt_en_quartznet15x5 (pretrained model)


In [ ]:
# pretrained model load (model : "stt_en_quartznet15x5")


## Small data를 통한 모델 확인 (영어)

- `datasets.load_dataset` : 학습 및 테스트 데이터 관리 라이브러리

In [ ]:
! pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
# LibriSpeech dataset load
english_ds = load_dataset("kresnik/librispeech_asr_test", "clean")

In [ ]:
english_ds

In [ ]:
# check file list
english_ds = english_ds['test']

In [ ]:
# check sample data
sample = english_ds[0]

In [ ]:
from pprint import pprint

In [ ]:
pprint(sample)

- `IPython.display` : IPython 위젯을 사용할 수 있는 라이브러리

In [ ]:
import IPython.display as ipd

In [ ]:
# listen audio file using ipd.Audio


In [ ]:
# transcribe using pretrained model
result =

In [ ]:
# check result
print(result)

In [ ]:
# compare with reference
print("hypothesis: " + result[0])
print("reference: " + sample['text'].lower())

- `jiwer` : CER, WER 등 음성인식 결과 평가 관련 라이브러리

In [ ]:
! pip install jiwer

In [ ]:
from jiwer import cer

In [ ]:
# calculate cer


In [ ]:
results = model.transcribe(english_ds['file'][:10])

## traning/test 데이터 불러오기 및 데이터 확인 (한국어)

- dataset : file, audio, text, speaker 정보

In [ ]:
# Zeroth-Korean dataset load
ds = load_dataset("kresnik/zeroth_korean", "clean")

In [ ]:
# check file list
ds

In [ ]:
total_train_ds =
test_ds =

In [ ]:
# check sample data
total_train_ds

In [ ]:
# listen audio file using ipd.Audio

In [ ]:
# split train, validation, test set
total_train_ds =

In [ ]:
total_train_ds

In [ ]:
train_ds = total_train_ds['train']
val_ds = total_train_ds['test']

In [ ]:
print(len(train_ds))
print(len(val_ds))
print(len(test_ds))

In [ ]:
train_ds.column_names

NeMo의 ASR data preparation은 3가지의 정보 필요
- audio_filepath, duration, text

In [ ]:
# remove columns ["audio", "speaker_id", "chapter_id", "id"]
train_ds =
val_ds =

In [ ]:
train_ds[0].keys()

In [ ]:
# rename column
train_ds = train_ds.rename_column(original_column_name= , new_column_name = )
val_ds = val_ds.rename_column(original_column_name= , new_column_name = )

In [ ]:
train_ds[0].keys()

In [ ]:
train_ds[0]

In [ ]:
import soundfile as sf

In [ ]:
# get duration

def get_duration(batch):


In [ ]:
get_duration(train_ds[0])

In [ ]:
train_ds = train_ds.map(get_duration)
val_ds = val_ds.map(get_duration)

In [ ]:
train_ds[0]

In [ ]:
ipd.Audio(train_ds[0]['audio_filepath'])

In [ ]:
import os

In [ ]:
train_json_path = os.path.abspath("train.json")
print(train_json_path)
val_json_path = os.path.abspath("validation.json")
print(val_json_path)

- `force_ascii = False` : 한글이 깨지지 않도록 encoding
- `orient = "records` : {columns:value} 형태의 딕셔너리를 요소로 하는 리스트 형태

In [ ]:
# make json file
train_ds.to_json(train_json_path, lines=True, force_ascii=False, orient="records")

In [ ]:
# check json file
! head -5 train.json

In [ ]:
# make validation json file
val_ds.to_json( )

In [ ]:
# check validation json file


In [ ]:
hug_ds = load_dataset("json", data_files=train_json_path)

In [ ]:
hug_ds = hug_ds['train']

In [ ]:
hug_ds[0]

In [ ]:
import pandas as pd

In [ ]:
pd.read_json()

## 한국어 fine-tuning을 위한 모델 설정
- 영어로 학습된 모델을 소용량의 한국어를 이용해 원활히 tuning하기 위해  
model의 encoder 정보는 그대로 유지  
model의 deocder 정보(sequence of output unit representation)을 재학습하는 것이 효율적)
- 충분한 양의 데이터를 확보하지 못했거나, computing 환경이 부족할 때 사용하는 방법
- encoder 전체를 학습하지 않는 경우 normalization 문제가 발생할 수 있음  
Ex> 원본 모델이 학습한 음성 데이터와 새로운 학습 데이터의 볼륨(소리 크기) 차이가 많이 나는 문제  
이를 방지하기 위해 batch normalization 부분은 freeze하지 않음




In [ ]:
import torch
import torch.nn as nn

In [ ]:
list(model.modules())

In [ ]:
# batch normalization unfreeze
def enable_bn(m):
  if type(m) == nn.BatchNorm1d:
    m.train()
    for param in m.parameters():
      param.requires_grad_(True)

In [ ]:
# encoder freeze


In [ ]:
for name, param in model.encoder.named_parameters():
  print(name, param.requires_grad)

In [ ]:
# unfreeze apply
model.encoder.apply(enable_bn)

In [ ]:
# unfreeze check (requires_grad -> True : 가중치 학습)
for name, param in model.encoder.named_parameters():
  print(name, param.requires_grad)

## 한국어 ouput unit 설정 및 training 세팅
- 영어 알파벳으로 정의된 모델을 한국어 음절로 변경

In [ ]:
# check model config
print(OmegaConf.to_yaml(model.cfg))

In [ ]:
train_ds[0]['text']

In [ ]:
#join


In [ ]:
# extract characters
def all_extract_characters(batch):


In [ ]:
len(train_ds)

In [ ]:
# make vocab list
train_vocab_list = train_ds.map(all_extract_characters, batched= True, batch_size=-1, remove_columns=train_ds.column_names)

In [ ]:
len(train_vocab_list["vocab"][0])

In [ ]:
val_vocab_list = val_ds.map(all_extract_characters, batched= True, batch_size=-1, remove_columns=val_ds.column_names)

In [ ]:
vocab_list = list(set(train_vocab_list["vocab"][0]) | set(val_vocab_list["vocab"][0]))

In [ ]:
print(len(vocab_list))

In [ ]:
# check vocab list

In [ ]:
# change model output unit
model.change_vocabulary(vocab_list)

In [ ]:
print(OmegaConf.to_yaml(model.cfg))

In [ ]:
import copy

In [ ]:
# copy configuration
print(OmegaConf.to_yaml(model.cfg.train_ds))

In [ ]:
cfg = copy.deepcopy(model.cfg)

In [ ]:
# setup train, validation configuration(manifest_filepath, labels, normalize_transcripts(대소문자), batch_size, num_workers, pin_memory, trim_silence)
with open_dict(cfg) as f:
  f.train_ds.manifest_filepath = train_json_path
  f.train_ds.normalize_transcripts = False
  f.train_ds.batch_size = 8
  f.train_ds.num_workers = 2
  f.train_ds.pin_memory = True
  f.train_ds.trim_silence = True

  f.validation_ds.manifest_filepath = val_json_path
  f.validation_ds.normalize_transcripts = False
  f.validation_ds.batch_size = 8
  f.validation_ds.num_workers = 2
  f.validation_ds.pin_memory = True
  f.validation_ds.trim_silence = True

In [ ]:
# setup data loader with new configs
model.setup_training_data(cfg.train_ds)
model.setup_validation_data(cfg.validation_ds)

In [ ]:
# print original optimizer + scheduler
print(OmegaConf.to_yaml(model.cfg.optim))

In [ ]:
??torch.optim.Adam

- lr = 0.01
- betas = [0.95, 0.25]
- weight_decay = 0.001 (original weight decay)
- sched.warup_steps = None (remove default number of steps of warmup)
- sched.warup_ratio = 0.05 (5% warmup)
- sched.min_lr = 1e-5

In [ ]:
# setup optimizer
with open_dict(model.cfg.optim) as f:
  f.betas = [0.95, 0.25]
  f.weight_decay = 0.001
  f.sched.warmup_steps = None
  f.sched.warmup_ratio = 0.05
  f.sched.min_lr = 1e-5

In [ ]:
print(OmegaConf.to_yaml(model.cfg.optim))

In [ ]:
# print original spec_augment
print(OmegaConf.to_yaml(model.cfg.spec_augment))

In [ ]:
model.spec_augmentaion = model.from_config_dict(model.cfg.spec_augment)

In [ ]:
print(OmegaConf.to_yaml(model.cfg.train_ds))

## 학습
- 학습 과정을 표시하는 metric 설정

In [ ]:
# use_cer & log_prediction
model._wer.use_cer = True
model._wer.log_prediction = True

In [ ]:
import torch
import pytorch_lightning as ptl

In [ ]:
if torch.cuda.is_available():
  gpus = 1
else:
  gpus = 0

In [ ]:
epochs = 1

In [ ]:
??ptl.Trainer

- gpus, max_epochs, accumulate_grad_batches, enable_checkpointing, logger, log_every_n_steps, check_val_every_n_epoch

In [ ]:
# set trainer
trainer = ptl.Trainer(
    gpus = gpus,
    max_epochs = True,
    accumulate_grad_batches = 1,
    enable_checkpointing = False,
    log_every_n_steps = 50,
    check_val_every_n_epoch=1
)

In [ ]:
# setup model with the trainer
model.set_trainer(trainer)

In [ ]:
# update the model's internal config
model.cfg = model._cfg

In [ ]:
import os

os.environ.pop('NEMO_EXPM_VERSION', None)

In [18]:
??exp_manager.ExpManagerConfig

In [ ]:
# set log path
config = exp_manager.ExpManagerConfig(
    exp_dir = f"experiment/lang/",
    name=f"ASR-Char-Model-Korean",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True
    ),
)

In [ ]:
config

In [ ]:
# set exp_manager
config =

In [ ]:
config

In [ ]:
logdir = exp_manager.exp_manager(trainer, config)

In [ ]:
# set tensorboard
%load_ext tensorboard
%tensorboard --logdir /content/experiment/lang/ASR-Char-Model-Korean/

In [ ]:
# start train


In [ ]:
# save model

## 테스트

In [ ]:
# load trained model

In [ ]:
# transcribe using trained model

In [ ]:
# compare with reference

In [ ]:
# calculate cer

## NeMo에서 full recipe로 학습을 수행하는 방법
- 임의의 데이터에 대해 같은 모델을 처음부터 학습하는 방법
- Colab에서는 동작이 어려움
- 시간 소요가 큼
- https://github.com/NVIDIA/NeMo/tree/main/examples/asr/asr_ctc